# Setup to allow tests in notebooks

Notice this setup uses:

- Azure Open AI
- Azure Search Service

The idea behind these Tests is to modulate the role of system to the required response.

Importante por leer:

- [Use your data AI search system message](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/use-your-data?tabs=ai-search#system-message)
- [System message templates](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/system-message)
- [Chain of thought prompting](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/advanced-prompt-engineering?pivots=programming-language-chat-completions#chain-of-thought-prompting)

In [1]:
import commons #Import commons methods and imports, please review commons.py

In [2]:
commons.print_required_variables(False)

No variables to print


Prueba de base, sin modificaciones

In [3]:
user_message = f"""\
Lista los beneficios de Baufest"""

messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 

response = commons.get_completion_from_messages(messages,show_completion=False)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Lista los beneficios de Baufest"
    }
]
Estimado colaborador, Baufest ofrece varios beneficios a sus empleados en diferentes áreas. A continuación, se detallan algunos de ellos:

- BAU Lunch (LATAM): Este beneficio ofrece a los empleados de Argentina, Perú y Chile la posibilidad de recibir acreditaciones para almuerzos a través de la App de PedidosYa o Tickets Restaurant emitidos por Edenred. La acreditación es semanal en Argentina y Perú, y mensual en Uruguay. [doc1]
- BAU Health (LATAM): Este beneficio permite a los empleados de América Latina solicitar el reembolso de actividades físicas realizadas en el trimestre correspondiente, siempre y cuando cumplan con los requisitos establecidos. [doc3]
- BAU Net (LATAM): Este beneficio ofrece a los empleados de América Latina la 

### Modifiquemos el prompt del _role user_ solamente

In [4]:
delimiter="####"
user_question=f"""\
Lista los beneficios de Baufest"""
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [5]:
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####Lista los beneficios de Baufest#### "
    }
]
Estimado colaborador, Baufest ofrece varios beneficios a sus empleados en LATAM, entre ellos se encuentran: BAU Lunch, BAU Health, BAU Net y BAU Dollar[doc1][doc2][doc3]. Cada uno de estos beneficios tiene diferentes características y requisitos, por lo que te recomiendo revisar los documentos correspondientes para obtener más información.


In [6]:
delimiter="####"
user_question=f"""\
Lista los beneficios de Baufest"""
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
\
Si no tienes los siguientes datos: Nombre, antigüedad del puesto, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario."""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [7]:
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####Lista los beneficios de Baufest#### Si no tienes los siguientes datos: Nombre, antig\u00fcedad del puesto, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario."
    }
]
Estimado colaborador, para poder responder tu pregunta de manera precisa, necesito que me proporciones algunos datos adicionales como tu nombre, antigüedad en el puesto, puesto y país de la unidad. Una vez que tenga esta información, podré brindarte una respuesta más detallada sobre los beneficios de Baufest. Quedo a la espera de tu respuesta.


In [8]:
user_question=f"""\
¿Cuál es el Beneficio de BAU lunch?"""

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
\
Si no tienes los siguientes datos: Nombre, antigüedad del puesto, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario."""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [9]:
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1l es el Beneficio de BAU lunch?#### Si no tienes los siguientes datos: Nombre, antig\u00fcedad del puesto, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario."
    }
]
Estimado colaborador, para poder responder a su pregunta sobre el Beneficio de BAU lunch, necesito que me proporcione su nombre, antigüedad del puesto, puesto y país de la unidad. Sin esta información, no puedo brindarle una respuesta precisa. Por favor, proporcione estos datos para que pueda ayudarlo mejor.


In [10]:
user_question=f"""\
¿Cuál es el Beneficio de BAU lunch?"""
user_name=""
user_seniority=""
user_position=""
user_country=""

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Teniendo en cuenta los siguientes datos:\
- Nombre :{user_name}\
- Antigüedad en el puesto : {user_seniority}\
- Puesto : {user_position}\
- País de la unidad : {user_country}\
\
Si no tienes los siguientes datos: Nombre, antigüedad en el puesto, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario."""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [11]:
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1l es el Beneficio de BAU lunch?#### Teniendo en cuenta los siguientes datos:- Nombre :- Antig\u00fcedad en el puesto : - Puesto : - Pa\u00eds de la unidad : Si no tienes los siguientes datos: Nombre, antig\u00fcedad en el puesto, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario."
    }
]
Estimado colaborador, para poder responder a tu pregunta sobre el Beneficio de BAU lunch, necesito que me proporciones los siguientes datos: Nombre, antigüedad en el puesto, puesto y país de la unidad. Sin esta información, no puedo brindarte una respuesta precisa. Por favor, proporciónamelos para poder ayudarte mejor.


In [12]:
user_question=f"""\
¿Cuál es el Beneficio de BAU lunch?"""
user_name="Alfonso Freddy"
user_seniority="5 años"
user_position="Desarrollador Full Stack"
user_country="Uruguay"

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Teniendo en cuenta los siguientes datos del colaborador:\
Nombre :{user_name}\
Antigüedad en el puesto : {user_seniority}\
Puesto : {user_position}\
País de la unidad : {user_country}\
\
Si no tienes los siguientes datos: Nombre, antigüedad en el puesto, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario."""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [13]:
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1l es el Beneficio de BAU lunch?#### Teniendo en cuenta los siguientes datos del colaborador:Nombre :Alfonso FreddyAntig\u00fcedad en el puesto : 5 a\u00f1osPuesto : Desarrollador Full StackPa\u00eds de la unidad : UruguaySi no tienes los siguientes datos: Nombre, antig\u00fcedad en el puesto, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario."
    }
]
Estimado colaborador, el beneficio de BAU Lunch en Uruguay consiste en una acreditación mensual que se reflejará en tu recibo de sueldo y que podrás utilizar para obtener tickets restaurant emitidos por Edenred[doc1][doc2]. Para aplicar al beneficio, es necesario cumplir con ciertos requisitos, como tener una antigüedad

In [ ]:
user_question=f"""\
¿Cuál es el Beneficio de BAU lunch?"""
user_name="Alejandra Escalante"
user_seniority="2 años"
user_position="Desarrollador Full Stack"
user_country="Argentina"

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Teniendo en cuenta los siguientes datos del colaborador:\
Nombre :{user_name}\
Antigüedad en el puesto : {user_seniority}\
Puesto : {user_position}\
País de la unidad : {user_country}\
\
Si no tienes los siguientes datos: Nombre, antigüedad en el puesto, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario.\
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [15]:
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1l es el Beneficio de BAU lunch?#### Teniendo en cuenta los siguientes datos del colaborador:Nombre :Alejandra EscalanteAntig\u00fcedad en el puesto : 2 a\u00f1osPuesto : Desarrollador Full StackPa\u00eds de la unidad : ArgentinaSi no tienes los siguientes datos: Nombre, antig\u00fcedad en el puesto, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario.Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"
    }
]
Estimada Alejandra Escalante, el beneficio de BAU Lunch consiste en un cupón mensual para almuerzos que podrás utilizar a través de la App de PedidosYa en Argentina. Para acceder a este beneficio, es necesario cumplir con los sigui

Después del refinamiento ajustamos el prompt usando los campos que se usarán como parte del contexto:

 - user_id
 - user_name
 - user_email
 - user_location
 - user_department
 - user_job
 - user_seniority


In [8]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name="Santiago Vilchis"
user_email="svilchis@baufest.com"
user_location="America/Mexico_City"
user_department="Software Development/SWDFIN"
user_job="Developer"
user_seniority="24 meses"

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Teniendo en cuenta los siguientes datos del colaborador: \
Nombre : {user_name} \
Puesto : {user_job} \
Departamento en el que trabaja : {user_department} \
Antigüedad en el puesto : {user_seniority} \
País en el que trabaja : {user_location} \
\
Si no tienes los siguientes datos: Nombre, Puesto, Departamento en el que trabaja, Antigüedad en el puesto y \
País en el que trabaja. Antes de responder solicita estos datos al usuario. \
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [9]:
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### Teniendo en cuenta los siguientes datos del colaborador: Nombre : Santiago Vilchis Puesto : Developer Departamento en el que trabaja : Software Development/SWDFIN Antig\u00fcedad en el puesto : 24 meses Pa\u00eds en el que trabaja : America/Mexico_City Si no tienes los siguientes datos: Nombre, Puesto, Departamento en el que trabaja, Antig\u00fcedad en el puesto y Pa\u00eds en el que trabaja. Antes de responder solicita estos datos al usuario. Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"
    }
]
Estimado Santiago Vilchis, según la información disponible en los documentos recuperados, Baufest ofrece varios beneficio

Parece ser que no está tomando en cuenta el país y otros aspectos

In [10]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name="Santiago Vilchis"
user_email="svilchis@baufest.com"
user_location="America/Mexico_City"
user_department="Software Development/SWDFIN"
user_job="Developer"
user_seniority="24 meses"

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Teniendo en cuenta los siguientes datos del colaborador: \
Nombre : {user_name} \
Puesto : {user_job} \
Departamento en el que trabaja : {user_department} \
Antigüedad en el puesto : {user_seniority} \
País en el que trabaja : {user_location} \
\
Si no tienes los siguientes datos: Nombre, Puesto, Departamento en el que trabaja, Antigüedad en el puesto y \
País en el que trabaja. Antes de responder solicita estos datos al usuario. \
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador en tu respuesta. \
Tu respuesta debe seguir el siguiente formato: \
Datos del usuario : #### <listar datos del colaborador> \
Repuesta : #### <Respuesta a la pregunta del colaborador>"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [12]:
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### Teniendo en cuenta los siguientes datos del colaborador: Nombre : Santiago Vilchis Puesto : Developer Departamento en el que trabaja : Software Development/SWDFIN Antig\u00fcedad en el puesto : 24 meses Pa\u00eds en el que trabaja : America/Mexico_City Si no tienes los siguientes datos: Nombre, Puesto, Departamento en el que trabaja, Antig\u00fcedad en el puesto y Pa\u00eds en el que trabaja. Antes de responder solicita estos datos al usuario. Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador en tu respuesta. Tu respuesta debe seguir el siguiente formato: Datos del usuario : #### <listar datos del colaborador> Repuesta :

In [13]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name="Santiago Vilchis"
user_email="svilchis@baufest.com"
user_location="America/Mexico_City"
user_department="Software Development/SWDFIN"
user_job="Developer"
user_seniority="24 meses"

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Teniendo en cuenta los siguientes datos del colaborador para tu respuesta: \
Nombre : {user_name} \
Puesto : {user_job} \
Departamento en el que trabaja : {user_department} \
Antigüedad en el puesto : {user_seniority} \
País en el que trabaja : {user_location} \
\
Si no tienes los siguientes datos: Nombre, Puesto, Departamento en el que trabaja, Antigüedad en el puesto y \
País en el que trabaja. Antes de responder solicita estos datos al usuario. \
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador en tu respuesta \
a la pregunta del colaborador. \
Tu respuesta debe seguir el siguiente formato: \
Datos del usuario : #### <listar datos del colaborador> \
Repuesta : #### <Respuesta a la pregunta del colaborador>"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [15]:
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### Teniendo en cuenta los siguientes datos del colaborador para tu respuesta: Nombre : Santiago Vilchis Puesto : Developer Departamento en el que trabaja : Software Development/SWDFIN Antig\u00fcedad en el puesto : 24 meses Pa\u00eds en el que trabaja : America/Mexico_City Si no tienes los siguientes datos: Nombre, Puesto, Departamento en el que trabaja, Antig\u00fcedad en el puesto y Pa\u00eds en el que trabaja. Antes de responder solicita estos datos al usuario. Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador en tu respuesta a la pregunta del colaborador. Tu respuesta debe seguir el siguiente formato: Datos del usuario 

La respuesta no fue tan satisfactoria retomemos el ejemplo que sí

In [16]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name="Alejandra Escalante"
user_seniority="2 años"
user_position="Desarrollador Full Stack"
user_country="Argentina"

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Teniendo en cuenta los siguientes datos del colaborador:\
Nombre :{user_name}\
Antigüedad en el puesto : {user_seniority}\
Puesto : {user_position}\
País de la unidad : {user_country}\
\
Si no tienes los siguientes datos: Nombre, antigüedad en el puesto, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario.\
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [18]:
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### Teniendo en cuenta los siguientes datos del colaborador:Nombre :Alejandra EscalanteAntig\u00fcedad en el puesto : 2 a\u00f1osPuesto : Desarrollador Full StackPa\u00eds de la unidad : ArgentinaSi no tienes los siguientes datos: Nombre, antig\u00fcedad en el puesto, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario.Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"
    }
]
Estimada Alejandra Escalante, Baufest ofrece varios beneficios para sus colaboradores en Argentina, entre ellos se encuentran: BAU Dollar, BAU Health, BAU Net y BAU Lunch. Para acceder a estos beneficios, es necesario cu

In [19]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name="Alejandra Escalante"
user_seniority="2 años"
user_job="Desarrollador Full Stack"
user_location="Argentina"

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Teniendo en cuenta los siguientes datos del colaborador:\
Nombre :{user_name}\
Antigüedad en Baufest : {user_seniority}\
Puesto : {user_job}\
País de la unidad : {user_location}\
\
Si no tienes los siguientes datos: Nombre, Antigüedad en Baufest, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario.\
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [22]:
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### Teniendo en cuenta los siguientes datos del colaborador:Nombre :Alejandra EscalanteAntig\u00fcedad en Baufest : 2 a\u00f1osPuesto : Desarrollador Full StackPa\u00eds de la unidad : ArgentinaSi no tienes los siguientes datos: Nombre, Antig\u00fcedad en Baufest, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario.Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"
    }
]
Estimada Alejandra Escalante, Baufest ofrece varios beneficios para sus colaboradores en Argentina, entre ellos se encuentran: 
- BAU Dollar: permite recibir una parte del sueldo en dólares, con la posibilidad de elegir la c

In [23]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name="Alejandra Escalante"
user_seniority="2 años"
user_job="Desarrollador Full Stack"
user_location="Argentina"

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Usa para tu respuesta los siguientes datos del colaborador: \
Nombre :{user_name} \
Antigüedad en Baufest : {user_seniority} \
Puesto : {user_job} \
País de la unidad : {user_location} \
\
Si no tienes los siguientes datos: Nombre, Antigüedad en Baufest, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario.\
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 


In [24]:
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### Usa para tu respuesta los siguientes datos del colaborador: Nombre :Alejandra Escalante Antig\u00fcedad en Baufest : 2 a\u00f1os Puesto : Desarrollador Full Stack Pa\u00eds de la unidad : Argentina Si no tienes los siguientes datos: Nombre, Antig\u00fcedad en Baufest, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario.Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"
    }
]
Lo siento, pero la pregunta delimitada por 4 hashtags no está clara. ¿Podría proporcionar más detalles o reformular la pregunta? Además, para poder responder a su pregunta, necesitaría saber si hay algún beneficio es

In [25]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name="Alejandra Escalante"
user_seniority="2 años"
user_job="Desarrollador Full Stack"
user_location="Argentina"

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Usa para tu respuesta los siguientes datos del colaborador: \
Nombre :{user_name} \
Antigüedad en Baufest : {user_seniority} \
Puesto : {user_job} \
País de la unidad : {user_location} \
\
Si no tienes los siguientes datos: Nombre, Antigüedad en Baufest, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario.\
Si no está clara la pregunta, responde de manera más flexible y solicita mayores detalles al colaborador. \
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### Usa para tu respuesta los siguientes datos del colaborador: Nombre :Alejandra Escalante Antig\u00fcedad en Baufest : 2 a\u00f1os Puesto : Desarrollador Full Stack Pa\u00eds de la unidad : Argentina Si no tienes los siguientes datos: Nombre, Antig\u00fcedad en Baufest, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario.Si no est\u00e1 clara la pregunta, responde de manera m\u00e1s flexible y solicita mayores detalles al colaborador. Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"
    }
]
Estimada Alejandra Escalante, como colaboradora de Baufest con una antigüedad de 2 años y como desar

In [26]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name="Alejandra Escalante"
user_seniority=""
user_job=""
user_location=""

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Usa para tu respuesta los siguientes datos del colaborador: \
Nombre :{user_name} \
Antigüedad en Baufest : {user_seniority} \
Puesto : {user_job} \
País de la unidad : {user_location} \
\
Si no tienes los siguientes datos: Nombre, Antigüedad en Baufest, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario.\
Si no está clara la pregunta, responde de manera más flexible y solicita mayores detalles al colaborador. \
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### Usa para tu respuesta los siguientes datos del colaborador: Nombre :Alejandra Escalante Antig\u00fcedad en Baufest :  Puesto :  Pa\u00eds de la unidad :  Si no tienes los siguientes datos: Nombre, Antig\u00fcedad en Baufest, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario.Si no est\u00e1 clara la pregunta, responde de manera m\u00e1s flexible y solicita mayores detalles al colaborador. Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"
    }
]
Estimada Alejandra Escalante, los beneficios que ofrece Baufest varían según la unidad de país en la que te encuentres. Algunos de los beneficio

In [27]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name="Alejandra Escalante"
user_seniority=""
user_job=""
user_location=""

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Usa para tu respuesta los siguientes datos del colaborador: \
Nombre :{user_name} \
Antigüedad en Baufest : {user_seniority} \
Puesto : {user_job} \
País de la unidad : {user_location} \
\
Si no tienes todos los siguientes datos: Nombre, Antigüedad en Baufest, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario.\
Si no está clara la pregunta, responde de manera más flexible y solicita mayores detalles al colaborador. \
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### Usa para tu respuesta los siguientes datos del colaborador: Nombre :Alejandra Escalante Antig\u00fcedad en Baufest :  Puesto :  Pa\u00eds de la unidad :  Si no tienes todos los siguientes datos: Nombre, Antig\u00fcedad en Baufest, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario.Si no est\u00e1 clara la pregunta, responde de manera m\u00e1s flexible y solicita mayores detalles al colaborador. Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"
    }
]
Estimado Alejandra Escalante, en relación a los beneficios que ofrece Baufest, puedo mencionar algunos de ellos según los documentos recup

In [29]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name=""
user_seniority=""
user_job=""
user_location=""

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Usa para tu respuesta los siguientes datos del colaborador: \
Nombre : {user_name} \
Antigüedad en Baufest : {user_seniority} \
Puesto : {user_job} \
País de la unidad : {user_location} \
\
Si no tienes todos los siguientes datos: Nombre, Antigüedad en Baufest, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario.\
Si no está clara la pregunta, responde de manera más flexible y solicita mayores detalles al colaborador. \
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### Usa para tu respuesta los siguientes datos del colaborador: Nombre :  Antig\u00fcedad en Baufest :  Puesto :  Pa\u00eds de la unidad :  Si no tienes todos los siguientes datos: Nombre, Antig\u00fcedad en Baufest, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario.Si no est\u00e1 clara la pregunta, responde de manera m\u00e1s flexible y solicita mayores detalles al colaborador. Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"
    }
]
Estimado colaborador, los beneficios que ofrece Baufest varían según el país de la unidad y la antigüedad en la empresa. Algunos de los beneficios que se of

In [31]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name=""
user_seniority=""
user_job=""
user_location=""

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags, i.e. {delimiter}. \
{delimiter}{user_question}{delimiter} \
Usa para tu respuesta los siguientes datos del colaborador: \
Nombre : {user_name} \
Antigüedad en Baufest : {user_seniority} \
Puesto : {user_job} \
País de la unidad : {user_location} \
\
Si no tienes todos los siguientes datos: Nombre, Antigüedad en Baufest, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario.\
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags, i.e. ####. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### Usa para tu respuesta los siguientes datos del colaborador: Nombre :  Antig\u00fcedad en Baufest :  Puesto :  Pa\u00eds de la unidad :  Si no tienes todos los siguientes datos: Nombre, Antig\u00fcedad en Baufest, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario.Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"
    }
]
Lo siento, pero la pregunta no está claramente delimitada por 4 hashtags. ¿Podría reformular su pregunta? Además, necesito los siguientes datos del colaborador para poder responder adecuadamente: Nombre, Antigüedad en Baufest, puesto y país de la unidad.


In [33]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name=""
user_seniority=""
user_job=""
user_location=""

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags. \
{delimiter}{user_question}{delimiter} \
Usa para tu respuesta los siguientes datos del colaborador: \
Nombre : {user_name} \
Antigüedad en Baufest : {user_seniority} \
Puesto : {user_job} \
País de la unidad : {user_location} \
\
Si no tienes todos los siguientes datos: Nombre, Antigüedad en Baufest, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario.\
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### Usa para tu respuesta los siguientes datos del colaborador: Nombre :  Antig\u00fcedad en Baufest :  Puesto :  Pa\u00eds de la unidad :  Si no tienes todos los siguientes datos: Nombre, Antig\u00fcedad en Baufest, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario.Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"
    }
]
Lo siento, pero la pregunta delimitada por 4 hashtags no es clara y específica. ¿Podría proporcionar más detalles o reformular su pregunta? Además, como asistente de recursos humanos, puedo proporcionar información sobre los beneficios que ofrece Baufest a sus colaboradores, pero ne

In [35]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name=""
user_seniority=""
user_job=""
user_location=""

delimiter="####"
user_message = f"""\
Responde la pregunta delimitada por 4 hashtags. \
{delimiter}{user_question}{delimiter} \
Teniendo en cuenta para tu respuesta los siguientes datos del colaborador: \
Nombre : {user_name} \
Antigüedad en Baufest : {user_seniority} \
Puesto : {user_job} \
País de la unidad : {user_location} \
\
Si no tienes todos los siguientes datos: Nombre, Antigüedad en Baufest, puesto y país de la unidad, antes de responder \
solicita estos datos al usuario.\
Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Responde la pregunta delimitada por 4 hashtags. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### Teniendo en cuenta para tu respuesta los siguientes datos del colaborador: Nombre :  Antig\u00fcedad en Baufest :  Puesto :  Pa\u00eds de la unidad :  Si no tienes todos los siguientes datos: Nombre, Antig\u00fcedad en Baufest, puesto y pa\u00eds de la unidad, antes de responder solicita estos datos al usuario.Si tienes los datos, usa el nombre del colaborador en lugar de estimado colaborador"
    }
]
Lo siento, pero la pregunta es demasiado amplia y no puedo proporcionar una respuesta precisa sin más información. ¿Podría proporcionar más detalles sobre los beneficios específicos que le interesan? Además, necesito conocer el nombre, la antigüedad en Baufest, el pue

In [37]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name=""
user_seniority=""
user_job=""
user_location=""

delimiter="####"

user_message = f"""\
Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: {delimiter}. \
Paso 1 : Ten en cuenta para tu respuesta los datos del colaborador. \
Paso 2 : Si no tienes los datos del colaborador, no respondas y en su lugar solicita dichos datos. \
Paso 3 : Si tienes los datos del colaborador, use en tu respuesta el nombre del colaborador \
en lugar de estimado colaborador. \
\
Datos del colaborador: \
Nombre : {user_name} \
Antigüedad en Baufest : {user_seniority} \
Puesto : {user_job} \
País de la unidad : {user_location} \
\
{delimiter}{user_question}{delimiter} \
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: ####. Paso 1 : Ten en cuenta para tu respuesta los datos del colaborador. Paso 2 : Si no tienes los datos del colaborador, no respondas y en su lugar solicita dichos datos. Paso 3 : Si tienes los datos del colaborador, use en tu respuesta el nombre del colaborador en lugar de estimado colaborador. Datos del colaborador: Nombre :  Antig\u00fcedad en Baufest :  Puesto :  Pa\u00eds de la unidad :  ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### "
    }
]
Estimado colaborador, Baufest ofrece varios beneficios para sus empleados en diferentes áreas. Algunos de estos beneficios incluyen el programa de salud BAU HEALTH, el programa de almuerzos BAU LUNCH, el programa de transporte ANTI 

In [38]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name=""
user_seniority=""
user_job=""
user_location=""

delimiter="####"

user_message = f"""\
Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: {delimiter}. \
Paso 1 : Si no tienes los datos del colaborador, no respondas y en su lugar solicita dichos datos. \
Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador \
en lugar de estimado colaborador. \
\
Datos del colaborador: \
Nombre : {user_name} \
Antigüedad en Baufest : {user_seniority} \
Puesto : {user_job} \
País de la unidad : {user_location} \
\
{delimiter}{user_question}{delimiter} \
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: ####. Paso 1 : Si no tienes los datos del colaborador, no respondas y en su lugar solicita dichos datos. Paso 2 : Si tienes los datos del colaborador, use en tu respuesta el nombre del colaborador en lugar de estimado colaborador. Datos del colaborador: Nombre :  Antig\u00fcedad en Baufest :  Puesto :  Pa\u00eds de la unidad :  ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### "
    }
]
Lo siento, para responder a tu pregunta necesito algunos datos del colaborador, como su nombre, antigüedad en Baufest, puesto y país de la unidad. Por favor, proporciónamelos para poder ayudarte mejor.


In [40]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name=""
user_seniority=""
user_job=""
user_location=""

delimiter="####"
no_user_data_prefix="Estimado Baufesiano para responder su pregunta necesito los siguientes datos"

user_message = f"""\
Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: {delimiter}. \
Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : {no_user_data_prefix} \
listando los datos que te falten y nada más. \
Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador \
en lugar de estimado colaborador. \
\
Datos del colaborador: \
Nombre : {user_name} \
Antigüedad en Baufest : {user_seniority} \
Puesto : {user_job} \
País de la unidad : {user_location} \
\
{delimiter}{user_question}{delimiter} \
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: ####. Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : Estimado Baufesiano para responder su pregunta necesito los siguientes datos listando los datos que te falten y nada m\u00e1s. Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador en lugar de estimado colaborador. Datos del colaborador: Nombre :  Antig\u00fcedad en Baufest :  Puesto :  Pa\u00eds de la unidad :  ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### "
    }
]
Estimado Baufesiano, para responder su pregunta necesito los siguientes datos: ¿En qué país trabaja? y ¿Cuál es su puesto en Baufest? Una vez que tenga esta información, podré

In [41]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name=""
user_seniority=""
user_job="Desarrollador"
user_location="Argentina"

delimiter="####"
no_user_data_prefix="Estimado Baufesiano para responder su pregunta necesito los siguientes datos"

user_message = f"""\
Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: {delimiter}. \
Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : {no_user_data_prefix} \
listando los datos que te falten y nada más. \
Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador \
en lugar de estimado colaborador. \
\
Datos del colaborador: \
Nombre : {user_name} \
Antigüedad en Baufest : {user_seniority} \
Puesto : {user_job} \
País de la unidad : {user_location} \
\
{delimiter}{user_question}{delimiter} \
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: ####. Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : Estimado Baufesiano para responder su pregunta necesito los siguientes datos listando los datos que te falten y nada m\u00e1s. Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador en lugar de estimado colaborador. Datos del colaborador: Nombre :  Antig\u00fcedad en Baufest :  Puesto : Desarrollador Pa\u00eds de la unidad : Argentina ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### "
    }
]
Estimado Baufesiano, para responder su pregunta necesito los siguientes datos: ¿A qué tipo de beneficios se refiere? Baufest ofrece varios beneficios par

In [45]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name=""
user_seniority=""
user_job="Desarrollador"
user_location="Argentina"

delimiter="####"
no_user_data_prefix="Estimado Baufesiano para responder su pregunta necesito los siguientes datos"

user_message = f"""\
Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: {delimiter}. \
Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : {no_user_data_prefix} \
listando los datos que te falten y nada más. \
Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador \
en lugar de estimado colaborador. \
Paso 3 : Usa las siguientes recomendaciones para tu respuesta : \
Recomendación 1 : Si la respuesta es muy amplia agrega en tu respuesta que te provea mayor detalle. \
\
Datos del colaborador: \
Nombre : {user_name}. \
Antigüedad en Baufest : {user_seniority}. \
Puesto : {user_job}. \
País de la unidad : {user_location}. \
\
{delimiter}{user_question}{delimiter} \
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: ####. Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : Estimado Baufesiano para responder su pregunta necesito los siguientes datos listando los datos que te falten y nada m\u00e1s. Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador en lugar de estimado colaborador. Paso 3 : Usa las siguientes recomendaciones para tu respuesta : Recomendaci\u00f3n 1 : Si la respuesta es muy amplia agrega en tu respuesta que te provea mayor detalle. Datos del colaborador: Nombre : . Antig\u00fcedad en Baufest : . Puesto : Desarrollador. Pa\u00eds de la unidad : Argentina. ####\u00bfCu\u00e1les son los beneficios que ofrece Bau

In [60]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name=""
user_seniority=""
user_job="Desarrollador"
user_location="Argentina"

delimiter="####"
no_user_data_prefix="Estimado Baufesiano para responder su pregunta necesito los siguientes datos"

user_message = f"""\
Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: {delimiter}. \
Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : {no_user_data_prefix} \
listando los datos que te falten y nada mas. \
Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador \
en lugar de estimado colaborador. \
Paso 3 : Usa las siguientes recomendaciones para tu respuesta : \
Recomendacion 1 : Si la respuesta es muy amplia, agrega a tu respuesta la solicitud de mayor detalle. \
\
Datos del colaborador: \
Nombre : {user_name}. \
Antiguedad en Baufest : {user_seniority}. \
Puesto : {user_job}. \
Pais de la unidad : {user_location}. \
\
{delimiter}{user_question}{delimiter} \
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: ####. Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : Estimado Baufesiano para responder su pregunta necesito los siguientes datos listando los datos que te falten y nada mas. Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador en lugar de estimado colaborador. Paso 3 : Usa las siguientes recomendaciones para tu respuesta : Recomendacion 1 : Si la respuesta es muy amplia, agrega a tu respuesta la solicitud de mayor detalle. Datos del colaborador: Nombre : . Antiguedad en Baufest : . Puesto : Desarrollador. Pais de la unidad : Argentina. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufest?#### "
    }


In [61]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name="Santiago Vilche"
user_seniority=""
user_job="Desarrollador"
user_location="Argentina"

delimiter="####"
no_user_data_prefix="Estimado Baufesiano para responder su pregunta necesito los siguientes datos"

user_message = f"""\
Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: {delimiter}. \
Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : {no_user_data_prefix} \
listando los datos que te falten y nada mas. \
Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador \
en lugar de estimado colaborador. \
Paso 3 : Usa las siguientes recomendaciones para tu respuesta : \
Recomendacion 1 : Si la respuesta es muy amplia, agrega a tu respuesta la solicitud de mayor detalle. \
\
Datos del colaborador: \
Nombre : {user_name}. \
Antiguedad en Baufest : {user_seniority}. \
Puesto : {user_job}. \
Pais de la unidad : {user_location}. \
\
{delimiter}{user_question}{delimiter} \
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: ####. Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : Estimado Baufesiano para responder su pregunta necesito los siguientes datos listando los datos que te falten y nada mas. Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador en lugar de estimado colaborador. Paso 3 : Usa las siguientes recomendaciones para tu respuesta : Recomendacion 1 : Si la respuesta es muy amplia, agrega a tu respuesta la solicitud de mayor detalle. Datos del colaborador: Nombre : Santiago Vilche. Antiguedad en Baufest : . Puesto : Desarrollador. Pais de la unidad : Argentina. ####\u00bfCu\u00e1les son los beneficios que ofrece Baufes

In [67]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name="Santiago Vilche"
user_seniority="2 años"
user_job="Desarrollador"
user_location="Argentina"

delimiter="####"
no_user_data_prefix="Estimado Baufesiano para responder su pregunta necesito los siguientes datos"

user_message = f"""\
Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: {delimiter}. \
Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : {no_user_data_prefix} \
listando los datos que te falten y nada mas. \
Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador \
en lugar de estimado colaborador. \
Paso 3 : Usa las siguientes recomendaciones para tu respuesta : \
Recomendacion 1 : Si la respuesta es muy amplia, agrega a tu respuesta la solicitud de mayor detalle. \
\
Datos del colaborador: \
Nombre : {user_name}. \
Antiguedad en Baufest : {user_seniority}. \
Puesto : {user_job}. \
Pais de la unidad : {user_location}. \
\
{delimiter}{user_question}{delimiter} \
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: ####. Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : Estimado Baufesiano para responder su pregunta necesito los siguientes datos listando los datos que te falten y nada mas. Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador en lugar de estimado colaborador. Paso 3 : Usa las siguientes recomendaciones para tu respuesta : Recomendacion 1 : Si la respuesta es muy amplia, agrega a tu respuesta la solicitud de mayor detalle. Datos del colaborador: Nombre : Santiago Vilche. Antiguedad en Baufest : 2 a\u00f1os. Puesto : Desarrollador. Pais de la unidad : Argentina. ####\u00bfCu\u00e1les son los beneficios que of

In [69]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name="Santiago Vilche"
user_seniority="2 años"
user_job="Desarrollador"
user_location="Argentina"

delimiter="####"
no_user_data_prefix="Estimado Baufesiano para responder su pregunta necesito los siguientes datos"

user_message = f"""\
Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: {delimiter}. \
Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : {no_user_data_prefix} \
listando los datos que te falten y nada mas. \
Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador \
en lugar de estimado colaborador. \
Paso 3 : Usa las siguientes recomendaciones para tu respuesta : \
Recomendacion 1 : Usa los datos del colaborador para filtrar la respuesta. \
Recomendacion 2 : Solo muestra los resultados que tengan que ver con el pais de la unidad y la antiguedad en Baufest. \
Recomendacion 3 : Si la respuesta es muy amplia, filtra la respuesta por el pais de la unidad y la antiguedad en Baufest. \
Recomendacion 4 : Si la pregunta no es clara, solicita mayor detalle para responder. \
\
Datos del colaborador: \
Nombre : {user_name}. \
Antiguedad en Baufest : {user_seniority}. \
Puesto : {user_job}. \
Pais de la unidad : {user_location}. \
\
{delimiter}{user_question}{delimiter} \
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: ####. Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : Estimado Baufesiano para responder su pregunta necesito los siguientes datos listando los datos que te falten y nada mas. Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador en lugar de estimado colaborador. Paso 3 : Usa las siguientes recomendaciones para tu respuesta : Recomendacion 1 : Usa los datos del colaborador para filtrar la respuesta. Recomendacion 2 : Solo muestra los resultados que tengan que ver con el pais de la unidad y la antiguedad en Baufest. Recomendacion 3 : Si la respuesta es muy amplia, filtra la respuesta por el pais de la unidad y l

In [70]:
user_question=f"""\
¿Cuáles son los beneficios que ofrece Baufest?"""
user_name="Santiago Vilche"
user_seniority="2 años"
user_job="Desarrollador"
user_location="Mexico"

delimiter="####"
no_user_data_prefix="Estimado Baufesiano para responder su pregunta necesito los siguientes datos"

user_message = f"""\
Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: {delimiter}. \
Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : {no_user_data_prefix} \
listando los datos que te falten y nada mas. \
Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador \
en lugar de estimado colaborador. \
Paso 3 : Usa las siguientes recomendaciones para tu respuesta : \
Recomendacion 1 : Usa los datos del colaborador para filtrar la respuesta. \
Recomendacion 2 : Solo muestra los resultados que tengan que ver con el pais de la unidad y la antiguedad en Baufest. \
Recomendacion 3 : Si la respuesta es muy amplia, filtra la respuesta por el pais de la unidad y la antiguedad en Baufest. \
Recomendacion 4 : Si la pregunta no es clara, solicita mayor detalle para responder. \
\
Datos del colaborador: \
Nombre : {user_name}. \
Antiguedad en Baufest : {user_seniority}. \
Puesto : {user_job}. \
Pais de la unidad : {user_location}. \
\
{delimiter}{user_question}{delimiter} \
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: ####. Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : Estimado Baufesiano para responder su pregunta necesito los siguientes datos listando los datos que te falten y nada mas. Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador en lugar de estimado colaborador. Paso 3 : Usa las siguientes recomendaciones para tu respuesta : Recomendacion 1 : Usa los datos del colaborador para filtrar la respuesta. Recomendacion 2 : Solo muestra los resultados que tengan que ver con el pais de la unidad y la antiguedad en Baufest. Recomendacion 3 : Si la respuesta es muy amplia, filtra la respuesta por el pais de la unidad y l

In [71]:
user_question=f"""\
¿Cuál es el beneficio de BauDollar?"""
user_name="Santiago Vilche"
user_seniority="2 años"
user_job="Desarrollador"
user_location="Mexico"

delimiter="####"
no_user_data_prefix="Estimado Baufesiano para responder su pregunta necesito los siguientes datos"

user_message = f"""\
Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: {delimiter}. \
Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : {no_user_data_prefix} \
listando los datos que te falten y nada mas. \
Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador \
en lugar de estimado colaborador. \
Paso 3 : Usa las siguientes recomendaciones para tu respuesta : \
Recomendacion 1 : Usa los datos del colaborador para filtrar la respuesta. \
Recomendacion 2 : Solo muestra los resultados que tengan que ver con el pais de la unidad y la antiguedad en Baufest. \
Recomendacion 3 : Si la respuesta es muy amplia, filtra la respuesta por el pais de la unidad y la antiguedad en Baufest. \
Recomendacion 4 : Si la pregunta no es clara, solicita mayor detalle para responder. \
\
Datos del colaborador: \
Nombre : {user_name}. \
Antiguedad en Baufest : {user_seniority}. \
Puesto : {user_job}. \
Pais de la unidad : {user_location}. \
\
{delimiter}{user_question}{delimiter} \
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: ####. Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : Estimado Baufesiano para responder su pregunta necesito los siguientes datos listando los datos que te falten y nada mas. Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador en lugar de estimado colaborador. Paso 3 : Usa las siguientes recomendaciones para tu respuesta : Recomendacion 1 : Usa los datos del colaborador para filtrar la respuesta. Recomendacion 2 : Solo muestra los resultados que tengan que ver con el pais de la unidad y la antiguedad en Baufest. Recomendacion 3 : Si la respuesta es muy amplia, filtra la respuesta por el pais de la unidad y l

In [72]:
user_question=f"""\
¿Qué opinas de la vida?"""
user_name="Santiago Vilche"
user_seniority="2 años"
user_job="Desarrollador"
user_location="Argentina"

delimiter="####"
no_user_data_prefix="Estimado Baufesiano para responder su pregunta necesito los siguientes datos"

user_message = f"""\
Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: {delimiter}. \
Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : {no_user_data_prefix} \
listando los datos que te falten y nada mas. \
Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador \
en lugar de estimado colaborador. \
Paso 3 : Usa las siguientes recomendaciones para tu respuesta : \
Recomendacion 1 : Usa los datos del colaborador para filtrar la respuesta. \
Recomendacion 2 : Solo muestra los resultados que tengan que ver con el pais de la unidad y la antiguedad en Baufest. \
Recomendacion 3 : Si la respuesta es muy amplia, filtra la respuesta por el pais de la unidad y la antiguedad en Baufest. \
Recomendacion 4 : Si la pregunta no es clara, solicita mayor detalle para responder. \
\
Datos del colaborador: \
Nombre : {user_name}. \
Antiguedad en Baufest : {user_seniority}. \
Puesto : {user_job}. \
Pais de la unidad : {user_location}. \
\
{delimiter}{user_question}{delimiter} \
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: ####. Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : Estimado Baufesiano para responder su pregunta necesito los siguientes datos listando los datos que te falten y nada mas. Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador en lugar de estimado colaborador. Paso 3 : Usa las siguientes recomendaciones para tu respuesta : Recomendacion 1 : Usa los datos del colaborador para filtrar la respuesta. Recomendacion 2 : Solo muestra los resultados que tengan que ver con el pais de la unidad y la antiguedad en Baufest. Recomendacion 3 : Si la respuesta es muy amplia, filtra la respuesta por el pais de la unidad y l

### ¿Funcionara usando el ejemplo de L4-Process-Inputs-Chain-of-Thought-Reasoning?

Usaremos la técnica de chain of though:

In [75]:
user_question=f"""\
¿Qué opinas de la vida?"""
user_name="Santiago Vilche"
user_seniority="2 años"
user_job="Desarrollador"
user_location="Argentina"

delimiter="####"
no_user_data_prefix="Estimado Baufesiano para responder su pregunta necesito los siguientes datos"

user_message = f"""\
Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: {delimiter}. \
Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : {no_user_data_prefix} \
listando los datos que te falten y nada mas. \
Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador \
en lugar de estimado colaborador. \
Paso 3 : Usa las siguientes recomendaciones para tu respuesta : \
Recomendacion 1 : Usa los datos del colaborador para filtrar la respuesta. \
Recomendacion 2 : Solo muestra los resultados que tengan que ver con el pais de la unidad y la antiguedad en Baufest. \
Recomendacion 3 : Si la respuesta es muy amplia, filtra la respuesta por el pais de la unidad y la antiguedad en Baufest. \
Recomendacion 4 : Si la pregunta no es clara, solicita mayor detalle para responder. \
\
Datos del colaborador: \
Nombre : {user_name}. \
Antiguedad en Baufest : {user_seniority}. \
Puesto : {user_job}. \
Pais de la unidad : {user_location}. \
\
{delimiter}{user_question}{delimiter} \
\
Usa el siguiente formato para tu respuesta : \
Paso 1: <Razonamiento paso 1> \
Paso 2: <Razonamiento paso 2> \
Paso 3: <Razonamiento paso 3> \
Respuesta al colaborador: <Respuesta a la pregunta> \
\
"""
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 
response = commons.get_completion_from_messages(messages)
print(response)

Data source role information : Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador
messages : [
    {
        "role": "user",
        "content": "Sigue los siguientes pasos para responder la pregunta delimitada por 4 hashtags: ####. Paso 1 : Si no tienes los datos del colaborador, tu respuesta debe seguir el siguiente formato : Estimado Baufesiano para responder su pregunta necesito los siguientes datos listando los datos que te falten y nada mas. Paso 2 : Si tienes los datos del colaborador, usa en tu respuesta el nombre del colaborador en lugar de estimado colaborador. Paso 3 : Usa las siguientes recomendaciones para tu respuesta : Recomendacion 1 : Usa los datos del colaborador para filtrar la respuesta. Recomendacion 2 : Solo muestra los resultados que tengan que ver con el pais de la unidad y la antiguedad en Baufest. Recomendacion 3 : Si la respuesta es muy amplia, filtra la respuesta por el pais de la unidad y l

Parece ser que tenemos que reformular la técnica de chain of thoughts

### Prompt en inglés

Ahora cambiaremos el prompt a inglés para ver si cambia la respuesta